# Q1. Getting the embeddings model

In [1]:
from sentence_transformers import SentenceTransformer

/Users/cristian.martinez/miniconda3/envs/llm-zoomcamp-env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model_name = "multi-qa-distilbert-cos-v1"
embedding_model = SentenceTransformer(model_name)

In [3]:
user_question = "I just discovered the course. Can I still join it?"
v = embedding_model.encode(user_question)
v[0]

0.078222595

## Prepare the documents

In [4]:
import requests 

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/documents-with-ids.json'
docs_url = f'{base_url}/{relative_url}?raw=1'
docs_response = requests.get(docs_url)
documents = docs_response.json()

In [5]:
ml_zoomcamp_documents = [doc for doc in documents if doc['course'] == "machine-learning-zoomcamp"]
len(ml_zoomcamp_documents)

375

# Q2. Creating the embeddings

In [6]:
import numpy as np
from tqdm.auto import tqdm

embeddings = []
documents_embeddings = []
for doc in tqdm(ml_zoomcamp_documents):
    question = doc["question"]
    text = doc["text"]
    qa_text = f"{question} {text}"
    embedding = embedding_model.encode(qa_text)
    embeddings.append(embedding)
    doc["qa_text_vector"] = embedding
    documents_embeddings.append(doc)

X = np.array(embeddings)


100%|██████████| 375/375 [00:31<00:00, 11.84it/s]


In [7]:
print(X.shape)

(375, 768)


# Q3. Search

In [8]:
scores = X.dot(v)

In [9]:
# get 5 highest scores in the vector scores
max(scores)

0.65065753

## Vector search

In [10]:
class VectorSearchEngine():
    def __init__(self, documents, embeddings):
        self.documents = documents
        self.embeddings = embeddings

    def search(self, v_query, num_results=10):
        scores = self.embeddings.dot(v_query)
        idx = np.argsort(-scores)[:num_results]
        return [self.documents[i] for i in idx]

search_engine = VectorSearchEngine(documents=ml_zoomcamp_documents, embeddings=X)
search_engine.search(v, num_results=5)

[{'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.',
  'section': 'General course-related questions',
  'question': 'The course has already started. Can I still join it?',
  'course': 'machine-learning-zoomcamp',
  'id': 'ee58a693',
  'qa_text_vector': array([ 8.06286111e-02, -6.66388497e-02,  2.52731014e-02, -1.30044539e-02,
          7.58728758e-02, -5.94626330e-02, -2.18839366e-02,  2.90002534e-03,
          7.92942359e-04, -5.22234850e-03, -3.36518027e-02, -2.79135406e-02,
          5.81165664e-02,  3.97482850e-02,  5.44185825e-02, -3.82516459e-02,
          6.30574450e-02, -3.89698707e-02,  4.33596957e-04, -3.55845802e-02,
          1.75637729e-03,  2.5

# Q4. Hit-rate for our search engine

In [11]:
import pandas as pd

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/ground-truth-data.csv'
ground_truth_url = f'{base_url}/{relative_url}?raw=1'

df_ground_truth = pd.read_csv(ground_truth_url)
df_ground_truth = df_ground_truth[df_ground_truth.course == 'machine-learning-zoomcamp']
ground_truth = df_ground_truth.to_dict(orient='records')

In [12]:
ground_truth[0]

{'question': 'Where can I sign up for the course?',
 'course': 'machine-learning-zoomcamp',
 'document': '0227b872'}

In [13]:
def hit_rate(relevance_total):
    cnt = 0

    for line in relevance_total:
        if True in line:
            cnt = cnt + 1

    return cnt / len(relevance_total)

def mrr(relevance_total):
    total_score = 0.0

    for line in relevance_total:
        for rank in range(len(line)):
            if line[rank] == True:
                total_score = total_score + 1 / (rank + 1)

    return total_score / len(relevance_total)

def evaluate(ground_truth, search_function):
    relevance_total = []

    for q in tqdm(ground_truth):
        doc_id = q['document']
        results = search_function(q)
        relevance = [d['id'] == doc_id for d in results]
        relevance_total.append(relevance)

    return {
        'hit_rate': hit_rate(relevance_total),
        'mrr': mrr(relevance_total),
    }

In [14]:
def knn_vector_search(q, k=5):
    question = q["question"]
    v = embedding_model.encode(question)
    return search_engine.search(v, num_results=k)

results = evaluate(ground_truth, search_function=knn_vector_search)

100%|██████████| 1830/1830 [00:58<00:00, 31.44it/s]


In [15]:
results["hit_rate"]

0.9398907103825137

# Q5. Indexing with Elasticsearch

In [16]:
from elasticsearch import Elasticsearch

es_client = Elasticsearch('http://localhost:9200') 
es_client.info()

ObjectApiResponse({'name': '93fd3d0839b5', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'NOqHMSYiSKu9vJP4jxfKzA', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [17]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"},
            "id": {"type": "keyword"},
            "qa_text_vector": {
                "type": "dense_vector",
                "dims": 768,
                "index": True,
                "similarity": "cosine"
            }
        }
    }
}

index_name = "ml-course-questions"

es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'ml-course-questions'})

In [18]:
# #created the dense vector using the pre-trained model
# operations = []
# for doc in documents:
#     # Transforming the title into an embedding using the model
#     doc["qa_text_vector"] = embedding_model.encode(f"{doc['question']} {doc['text']}").tolist()
#     operations.append(doc)

In [19]:
# operations[0]

In [20]:
for doc in documents_embeddings:
    try:
        es_client.index(index=index_name, document=doc)
    except Exception as e:
        print(e)

In [23]:
query = {
    "field": "qa_text_vector",
    "query_vector": v,
    "k": 5,
    "num_candidates": 10000, 
}

res = es_client.search(index=index_name, knn=query, source=["text", "section", "id", "question", "course"])
res["hits"]["hits"][0]

{'_index': 'ml-course-questions',
 '_id': 'khTWxZAB2ur2TIEueHz3',
 '_score': 0.8253288,
 '_source': {'question': 'The course has already started. Can I still join it?',
  'course': 'machine-learning-zoomcamp',
  'section': 'General course-related questions',
  'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.',
  'id': 'ee58a693'}}

# Q6. Hit-rate for Elasticsearch

In [24]:
def elastic_search_knn(vector, k=5):
    knn = {
        "field": "qa_text_vector",
        "query_vector": vector,
        "k": 5,
        "num_candidates": 10000,
    }

    search_query = {
        "knn": knn,
        "_source": ["text", "section", "question", "course", "id"]
    }

    es_results = es_client.search(
        index=index_name,
        body=search_query
    )
    
    result_docs = []
    
    for hit in es_results['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs

def question_vector_knn(q):
    question = q['question']

    v_q = embedding_model.encode(question)

    return elastic_search_knn(v_q)

In [25]:
results = evaluate(ground_truth, search_function=question_vector_knn)

100%|██████████| 1830/1830 [00:43<00:00, 42.08it/s]


In [26]:
results["hit_rate"]

0.9398907103825137